<a href="https://colab.research.google.com/github/AksleyRios/DataAquosmic/blob/main/PruebaPipeline/Prueba_de_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports/preparación

In [1]:
!pip install openpyxl --upgrade
!pip install utm
import pandas as pd
import numpy as np 
from datetime import datetime
import ee
import utm
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=ee181ce7933e7605b2b45c0c790d60e1642ec270a6c3b0cef5a272461c7690d0
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [2]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6aKi3kICMBaizfpyDhPKddYFhfynd0nm_XJwqNAgRnU&tc=Jzuh9OmtnANI9r4jLSse7CjXG5rOVXPTlHWa5K2uVCo&cc=qHSEUsEast5RFe1vHsW087KP3Av-IfvCyPrcACjE6SQ

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiNXWmu4mJjTxmR0Gqkly-lBY9Pu5aILxS_VNy2xXXScmLLDtqfEmE

Successfully saved authorization token.


In [3]:
ee.Initialize()

#Recopilación de datos

##Longitud, Latitud y Fecha

In [30]:
lonarr=[]
latarr=[]
datearr=[]
print("¿Cuántos puntos tienes?")
n=input()
for i in range(0,int(n)):
  print("Longitud:")
  lon=input()
  lonarr.append(float(lon))
  print("Latitud:")
  lat=input()
  latarr.append(float(lat))
  print("Día:")
  day=input()
  print("Mes:")
  month=input()
  print("Año (YYYY):")
  year=input()
  date=year+'-'+month+'-'+day
  print(date)
  date=datetime.strptime(date, '%Y-%m-%d')
  datearr.append(date)
lon_pd = pd.DataFrame(lonarr)
lon_pd.columns = ['Lon']
lat_pd = pd.DataFrame(latarr)
lat_pd.columns = ['Lat']
date_pd = pd.DataFrame(datearr)
date_pd.columns = ['Date']
Base=lon_pd.join(lat_pd)
Base=Base.join(date_pd)
Base

¿Cuántos puntos tienes?
1
Longitud:
-102.465
Latitud:
22.18899
Día:
28
Mes:
04
Año (YYYY):
2016
2016-04-28


,Lon,Lat,Date
0,-102.465,22.18899,2016-04-28


##Bandas

In [31]:
date_plus = []
data_1f = Base.reset_index()  # make sure indexes pair with number of rows


for index, row in data_1f.iterrows():
    date_row = row['Date']
    date_plus.append(date_row)


date_pd = pd.DataFrame(date_plus)
date_pd.columns = ['Date_ref']

data_1j = Base.join(date_pd)

data_1j['Date_plus'] = data_1j['Date_ref'] + pd.DateOffset(months=1)
data_1j['Date_less'] = data_1j['Date_ref'] + pd.DateOffset(months=-1)

data_1j.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Lon        1 non-null      float64       
 1   Lat        1 non-null      float64       
 2   Date       1 non-null      datetime64[ns]
 3   Date_ref   1 non-null      datetime64[ns]
 4   Date_plus  1 non-null      datetime64[ns]
 5   Date_less  1 non-null      datetime64[ns]
dtypes: datetime64[ns](4), float64(2)
memory usage: 176.0 bytes


In [32]:
B1 = []
B2 = []
B3 = []
B4 = []
B5 = []
B6 = []
B7 = []
B8 = []
B9 = []
B10 = []
B11 = []
ActDate = []

#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1')
for index, row in data_1j.iterrows():
  try:
    landsat_1f = landsat.filterDate(row['Date_less'],row['Date_plus'])
    Image_a = ee.Image(landsat_1f.filterBounds(ee.Geometry.Point([row['Lon'], row['Lat']])).sort('CLOUD_COVER').first())
    p = ee.Geometry.Point(row['Lon'], row['Lat'])
    meanDictionary = Image_a.reduceRegion(reducer=ee.Reducer.mean(), geometry=p, scale=1)
    u = meanDictionary.getInfo()
    B1.append(u['B1'])
    B2.append(u['B2'])
    B3.append(u['B3'])
    B4.append(u['B4'])
    B5.append(u['B5'])
    B6.append(u['B6'])
    B7.append(u['B7'])
    B8.append(u['B8'])
    B9.append(u['B9'])
    B10.append(u['B10'])
    B11.append(u['B11'])
    date1 = ee.Date(landsat_1f.filterBounds(ee.Geometry.Point([row['Lon'], row['Lat']])).sort('CLOUD_COVER').first().get('system:time_start'))
    date2 = date1.format('Y-MM-dd').getInfo()
    ActDate.append(date2)
    print(index)
  except:
    B1.append('')
    B2.append('')
    B3.append('')
    B4.append('')
    B5.append('')
    B6.append('')
    B7.append('')
    B8.append('')
    B9.append('')
    B10.append('')
    B11.append('')
    ActDate.append('')

b1_pd = pd.DataFrame(B1)
b1_pd.columns = ['B1']
b2_pd = pd.DataFrame(B2)
b2_pd.columns = ['B2']
b3_pd = pd.DataFrame(B3)
b3_pd.columns = ['B3']
b4_pd = pd.DataFrame(B4)
b4_pd.columns = ['B4']
b5_pd = pd.DataFrame(B5)
b5_pd.columns = ['B5']
b6_pd = pd.DataFrame(B6)
b6_pd.columns = ['B6']
b7_pd = pd.DataFrame(B7)
b7_pd.columns = ['B7']
b8_pd = pd.DataFrame(B8)
b8_pd.columns = ['B8']
b9_pd = pd.DataFrame(B9)
b9_pd.columns = ['B9']
b10_pd = pd.DataFrame(B10)
b10_pd.columns = ['B10']
b11_pd = pd.DataFrame(B11)
b11_pd.columns = ['B11']
actdate_pd = pd.DataFrame(ActDate)
actdate_pd.columns = ['Actual_Date']

data_2j = data_1j.join(actdate_pd)
data_2j = data_2j.join(b1_pd)
data_2j = data_2j.join(b2_pd)
data_2j = data_2j.join(b3_pd)
data_2j = data_2j.join(b4_pd)
data_2j = data_2j.join(b5_pd)
data_2j = data_2j.join(b6_pd)
data_2j = data_2j.join(b7_pd)
data_2j = data_2j.join(b8_pd)
data_2j = data_2j.join(b9_pd)
data_2j = data_2j.join(b10_pd)
data_2j = data_2j.join(b11_pd)

BaseLimpiaBandas=data_2j
BaseLimpiaBandas

0


,Lon,Lat,Date,Date_ref,Date_plus,Date_less,Actual_Date,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11
0,-102.465,22.18899,2016-04-28,2016-04-28,2016-05-28,2016-03-28,2016-05-17,11190,10822,10862,11626,14167,16506,13966,12407,5222,31434,28510


In [60]:
col=["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10", "B11"]
for i in col:
  BaseLimpiaBandas[i]=BaseLimpiaBandas[i].apply(lambda x: x/65455)
BaseLimpiaBandas

,Lon,Lat,Date,Date_ref,Date_plus,Date_less,Actual_Date,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11
0,-102.465,22.18899,2016-04-28,2016-04-28,2016-05-28,2016-03-28,2016-05-17,0.170957,0.165335,0.165946,0.177618,0.216439,0.252173,0.213368,0.18955,0.07978,0.480238,0.435566


#Predicciones

##Modelos

In [42]:
lrOUSA=pickle.load(open('LinearOxigenoUSA.sav','rb'))
lrNUSA=pickle.load(open('LinearNitrogenoUSA.sav','rb'))
lrFUSA=pickle.load(open('LinearFosforoUSA.sav','rb'))
lrCOUSA=pickle.load(open('LinearConductividadUSA.sav','rb'))
lrCLUSA=pickle.load(open('LinearClorofilaUSA.sav','rb'))
lrOMex=pickle.load(open('LinearOxigenoMéxicoLent.sav','rb'))
lrNMex=pickle.load(open('LinearNitrogenoMéxicoLent.sav','rb'))
lrCOMex=pickle.load(open('LinearConductividadMéxicoLent.sav','rb'))
lrTMex=pickle.load(open('LinearTurbiedadMéxicoLent.sav','rb'))
lrFMex=pickle.load(open('LinearFosforoMéxicoLent.sav','rb'))

In [59]:
OxiUSAArr=[]
OxiMexArr=[]
NitUSAArr=[]
NitMexArr=[]
CondUSAArr=[]
CondMexArr=[]
FosfUSAArr=[]
FosfMexArr=[]
ClorofUSAArr=[]
TurbMexArr=[]
for i in range(0,int(n)):
  OxiUSAArr.append(lrOUSA.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']].iloc[i]).reshape(1,-1)))
OxiUSAArr

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


[array([-314671.16145333])]

In [57]:
BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']]

,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11
0,11190,10822,10862,11626,14167,16506,13966,12407,5222,31434,28510


##Tabla

In [43]:
Modelo=['USA','México']*n


SyntaxError: ignored